# Test Predictions

Notebook to compute and store predictions on the operational test set.

Full-disk and patches/sector predictions are stored in separated files 

# Config

In [1]:
# !pip install tensorflow==2.15.0

In [2]:
COLAB = False

if COLAB : 
  configSetup = {
      'COLAB'           : 'True',
      'PATH_ROOT_DRIVE' : '/content/drive/MyDrive/Projects/Forecast',
      'PATH_ROOT_LOCAL' : '/content/session',
      'PATH_SUNDL'      : '/content/sundl',
      'PATH_PROJECT'    : '/content/flare_limits_pcnn'
  }
  !git clone https://github.com/gfrancisco20/sundl.git
  !git clone https://github.com/gfrancisco20/flare_limits_pcnn.git
  import sys
  import re
  sys.path.append(configSetup['PATH_SUNDL'])
  sys.path.append(configSetup['PATH_PROJECT'])
  configFile = f'{configSetup["PATH_PROJECT"]}/config.py'
  with open(configFile, 'r') as file:
    content = file.read()
  for constant in configSetup.keys():
    content = re.sub(re.compile(f'{constant} = .*'), f'{constant} = \'{configSetup[constant]}\'', content)
  with open(configFile, 'w') as file:
    file.write(content)
   
from config import *
if COLAB:
  from sundl.utils.colab import mountDrive
  # mouting drive content in session on colab
  mountDrive()

# Setup

In [3]:
FOLDER = PATH_RES/'Classification_PCNN_Limits_Paper_2024_03_21__0'# 'Results_Paper_PCNN' #

# Seleted Models from the CV resullts
modelDict = {
  # 'C+_mpf_Persistant_24'                                    : 'C+_Persistant',
  # 'C+_mpf_PTx8_RtdXall_ProgPos_AW4e6D4e3_blos_24'           : 'C+_SPCNN_Blos', #
  # 'C+_mpf_PTx8_RtdXall_ProgPos_AW1e5D1e4_0193x0211x0094_24' : 'C+_SPCNN_EUV',
  # 'M+_mpf_Persistant_24'                                    : 'M+_Persistant',
  # 'M+_mpf_PTx8_RtdXall_LowC_AW6e6D1e3_blos_24'              : 'M+_SPCNN_Blos',
  # 'M+_mpf_PTx8_RtdXall_ProgPos_AW1e5D1e4_0193x0211x0094_24' : 'M+_SPCNN_EUV',
  'C+_mpf_png_blos_24__PTx8_RtdXall_ProgPos_AW1e5D1e4' : 'C+_SPCNN_Blos'
}
modelDictRev = {modelDict[oldName] : oldName for oldName in modelDict.keys()}

ensembles = {'C+_SPCNN_Both_Max'  : ['C+_SPCNN_Blos','C+_SPCNN_EUV'],
             'M+_SPCNN_Both_Max'  : ['M+_SPCNN_Blos','M+_SPCNN_EUV'], 
             'C+_SPCNN_Both_Avg'  : ['C+_SPCNN_Blos','C+_SPCNN_EUV'],
             'M+_SPCNN_Both_Avg'  : ['M+_SPCNN_Blos','M+_SPCNN_EUV'] 
            #  'C+_SPCNN_Histo' : ['C+_SPCNN_Blos','C+_SPCNN_EUV','C+_Persistant'],
            #  'M+_SPCNN_Histo' : ['M+_SPCNN_Blos','M+_SPCNN_EUV', 'M+_Persistant']
             }

# Predictions

In [4]:
modelName

NameError: name 'modelName' is not defined

In [8]:
modelName.split('_')

['C+',
 'mpf',
 'png',
 'blos',
 '24',
 '',
 'PTx8',
 'RtdXall',
 'ProgPos',
 'AW1e5D1e4']

In [15]:
n_fold

1

In [17]:
modelsConfigAndPath

['/Users/greg/Google Drive/Mi unidad/Projects/Forecast/Results/Flare/Classification_PCNN_Limits_Paper_2024_03_21__0/models/C+_mpf_png_blos_24__PTx8_RtdXall_ProgPos_AW1e5D1e4_fd000.keras',
 '/Users/greg/Google Drive/Mi unidad/Projects/Forecast/Results/Flare/Classification_PCNN_Limits_Paper_2024_03_21__0/models/C+_mpf_png_blos_24__PTx8_RtdXall_ProgPos_AW1e5D1e4_fd000.keras_config.pkl']

In [19]:
[m for m in modelsConfigAndPath if m[-3:]!='pkl']

['/Users/greg/Google Drive/Mi unidad/Projects/Forecast/Results/Flare/Classification_PCNN_Limits_Paper_2024_03_21__0/models/C+_mpf_png_blos_24__PTx8_RtdXall_ProgPos_AW1e5D1e4_fd000.keras']

In [20]:
modelFoldsPath = [m for m in modelsConfigAndPath if m[-3:]!='pkl']
modelFoldsConfigs = [m for m in modelsConfigAndPath if m[-3:]=='pkl']
modelFoldsConfigs

['/Users/greg/Google Drive/Mi unidad/Projects/Forecast/Results/Flare/Classification_PCNN_Limits_Paper_2024_03_21__0/models/C+_mpf_png_blos_24__PTx8_RtdXall_ProgPos_AW1e5D1e4_fd000.keras_config.pkl']

In [28]:
modelFoldsPath

[]

In [29]:
modelsConfigAndPath

['/Users/greg/Google Drive/Mi unidad/Projects/Forecast/Results/Flare/Classification_PCNN_Limits_Paper_2024_03_21__0/models/C+_mpf_png_blos_24__PTx8_RtdXall_ProgPos_AW1e5D1e4_fd000.keras',
 '/Users/greg/Google Drive/Mi unidad/Projects/Forecast/Results/Flare/Classification_PCNN_Limits_Paper_2024_03_21__0/models/C+_mpf_png_blos_24__PTx8_RtdXall_ProgPos_AW1e5D1e4_fd000.keras_config.pkl']

In [5]:
%%time
import dill as pickle
from pathlib import Path
from glob import glob
import numpy as np
import tensorflow as tf

from sundl.utils.data import read_Dataframe_With_Dates

for modelName in modelDict.keys():
  # TBR
  try:
    if modelName.split('_')[2][3] in ['4']:
      # TODO : make generic to patch number.
      continue
  except:
    pass
    
  labelCol = modelName.split('_')[1]
  h = modelName.split('_')[-1]
  
  #@@@@@@@@
  print('WARNING : Rmv l20  !')
  h=24
  #@@@@@@@@

  pathPredFd = F_PATH_PREDS(FOLDER)/f'{modelName}_fd.csv'
  pathPredPt = F_PATH_PREDS(FOLDER)/f'{modelName}_pt.csv'
  if not pathPredFd.exists():
    print('\n\nMODEL : ',modelName)

    # loading model path and configs for availlable fold models
    modelsConfigAndPath = glob((FOLDER/f'models/{modelName}*').as_posix())
    modelFoldsPath = [m for m in modelsConfigAndPath if m[-3:]!='pkl']
    modelFoldsConfigs = [m for m in modelsConfigAndPath if m[-3:]=='pkl']
    # foldIds = [m[-5:] for m in modelFoldsPath]
    foldIds = [m.split('/')[-1].split('.')[0][-5:] for m in modelFoldsPath]
    n_fold = len(foldIds)
    if n_fold == 0:
      continue
    # for foldIdx in range(n_fold-1,-1,-1):
    #   if not Path(f'{modelFoldsPath[foldIdx]}/assets').exists():
    #     modelFoldsPath.pop(foldIdx)
    #     modelFoldsConfigs.pop(foldIdx)
    #     foldIds.pop(foldIdx)
    print('FOLDS WITH MODELS: ',foldIds)
    if len(foldIds)==0:
      continue
    with open(modelFoldsConfigs[0], 'rb') as f1:
      config = pickle.load(f1)
    configTest = config['dataset_val']
    dfTest = read_Dataframe_With_Dates(F_PATH_TEST(configTest['labelCol'],configTest['ts_off_label_hours'][0]))
    # configTest = config['dataset_val']
    configTest['dfId2History'] = dfTest.copy()
    configTest['samples'] = None
    configTest['cache'] = False
    configTest['shuffle'] = False
    configTest['weightByClass'] = False
    configTest['batch_size'] = 64
    configTest['epochs'] = 1
    # STOP
    # encoders consstantss **
    classTresholds = configTest['classTresholds']
    binCls = modelName[0]
    # **
    if modelName.split('_')[2] == 'Persistant':
      # dsTest , _, _, _ = build_dataset_persistant(**configTest)
      # saved pstmod not working
      dfId2History =  dfTest.copy()
      ts_off_label_hours = h
      ts_off_history_hours = -h
      dfId2History.index = dfId2History.index.shift(periods = -ts_off_label_hours, freq='H')
      input_lag = - ts_off_history_hours
      dfId2History['history'] = dfId2History[labelCol].rolling(window = f'{input_lag}H',
                                                      closed = 'right', # min_periods = int(input_lag)
                                                      ).apply(
                                                          lambda x: x[0]) # we remove first month in case of incomplete windows
      dfId2History = dfId2History[int(ts_off_label_hours/2):-int(ts_off_label_hours/2)]
      dfId2History = dfId2History.copy()

      dfId2History['pred'] = dfId2History['history'].apply(lambda x: configTest['labelEncoder'](x))
      dfId2History['label'] = dfId2History[labelCol].apply(lambda x: configTest['labelEncoder'](x))
      dfPred = dfId2History[[labelCol,'cls','label','pred']].copy()
      dfPred.to_csv(pathPredFd)
    else:
      if 'buildDsFunction' in config.keys():
        buildDs = config[f'buildDsFunction']
      else:
        # default dataset builder if not stored in config (adapt if needed)
        buildDs = builDS_image_feature
      dsTest , _, missing_file_regexp, dfSamples_corr = buildDs(**configTest)
      dfSamples_corr = dfSamples_corr.set_index('timestamp',drop = True)

      # predictionss
      predsFd = []
      patch_predsFd = []
      for foldIdx,modelFoldPath in enumerate(modelFoldsPath):
        # Full-disk predictions
        model = tf.keras.models.load_model(modelFoldPath, compile=False)
        
        STOP
        
        predsFd.append(model.predict(dsTest))
        # Patches predictions
        if modelName.split('_')[2][:2] == 'PT':
          for layer in model.layers:
            if layer.name == 'time_distributed':
              patchesBlock = layer
          patches = tf.keras.Model(model.input, patchesBlock.output, name='patches')
          patch_predsFd.append(patches.predict(dsTest))
          # predsFd could be retrieved from it to avoid double computation
          del patches
        del model
      del dsTest

      # predictions to dataframes
      dfPred = dfSamples_corr[[labelCol,'cls']].copy()
      dfPred['label'] = dfPred[labelCol].apply(lambda x: configTest['labelEncoder'](x))
      dfPred['pred'] = np.zeros(len(dfPred))

      for idx,foldId in enumerate(foldIds):
        dfPred[f'pred_{foldId}'] = predsFd[idx][:,1]
        dfPred['pred'] += dfPred[f'pred_{foldId}']
      dfPred['pred'] /= (idx+1)
      dfPred.to_csv(pathPredFd)

      if modelName.split('_')[2][:2] == 'PT':
        dfPredPatches = dfSamples_corr[[labelCol,'cls']].copy()
        dfPredPatches['label'] = dfPredPatches[labelCol].apply(lambda x: configTest['labelEncoder'](x))
        num_ptch = patch_predsFd[0].shape[1]
        for ptchId in range(num_ptch):
          dfPredPatches[f'pred_pt{ptchId}'] = np.zeros(len(dfPredPatches))
          for idx,foldId in enumerate(foldIds):
            dfPredPatches[f'pred_pt{ptchId}_{foldId}'] = patch_predsFd[idx][:,ptchId,0]
            dfPredPatches[f'pred_pt{ptchId}']  += dfPredPatches[f'pred_pt{ptchId}_{foldId}']
          dfPredPatches[f'pred_pt{ptchId}'] /= (idx+1)
          
# print('Ensemble models prediction')
# for modelName in ensembles.keys():
#   ensModIsMax = False
#   if modelName.split('_')[-1] in ['max','Max']:
#     ensModIsMax = True
#   pathPredFd =  F_PATH_PREDS(FOLDER)/f'{modelName}_fd.csv'
#   pathPredPt =  F_PATH_PREDS(FOLDER)/f'{modelName}_pt.csv'
#   if not pathPredPt.exists():
#     for pathPred in [pathPredFd,pathPredPt]:
#       predTag = pathPred.as_posix().split('_')[-1][:2]
#       for idx,subModelName in enumerate(ensembles[modelName]):
#         subModel = read_Dataframe_With_Dates(F_PATH_PREDS(FOLDER)/f'{modelDictRev[subModelName]}_{predTag}.csv')
#         if idx == 0:
#           dfPred = subModel.copy()
#           pred_cols = [col for col in dfPred.columns if col[:3]=='pre']
#           ptcPredCols = [col for col in dfPred.columns if len(col)==len('pred_pt0')]
#           num_ptc = len(ptcPredCols)
#         else:
#           # keeping only common dates
#           tmp = subModel.copy()
#           tmp = tmp[tmp.index.isin(dfPred.index)].sort_index()
#           dfPred = dfPred[dfPred.index.isin(tmp.index)].sort_index()
#           # averaging
#           if modelDictRev[subModelName].split('_')[2] == 'Persistant':
#             # empirical persistent probability
#             for ptchTag in [''] + [f'_pt{ptcId}' for ptcId in range(num_ptc)]:
#               c = tmp[f'change{ptchTag}'].sum()
#               tot = len(tmp)
#               pChange = c/ (2*tot)
#               tmp[tmp[f'histo{ptchTag}']==1][f'pred{ptchTag}'] = 1 - pChange # prob of positive event
#               tmp[tmp[f'histo{ptchTag}']==0][f'pred{ptchTag}'] = pChange # prob of positive event
#             for col in pred_cols:
#               if ensModIsMax:
#                 dfPred[col] = np.maximum(dfPred[col],tmp[col])
#               else:
#                 dfPred[col] = dfPred[col] + tmp[col]
#           else:
#             missingCols = [col for col in pred_cols if col not in tmp.columns]
#             for col in missingCols:
#               tmp[col] = dfPred[col]
#             if ensModIsMax:
#               for col in pred_cols:
#                 dfPred[col] = np.maximum(dfPred[col],tmp[col])
#             else:
#               dfPred[pred_cols] = dfPred[pred_cols] + tmp[pred_cols]
#       if not ensModIsMax:
#         dfPred[pred_cols] = dfPred[pred_cols]/len(ensembles[modelName])
#       dfPred = dfPred[['mpf','cls','label']+[col for col in dfPred.columns if col[:3]=='pre']]
#       dfPred.to_csv(pathPred)


2024-03-21 17:22:55.826629: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.




MODEL :  C+_mpf_png_blos_24__PTx8_RtdXall_ProgPos_AW1e5D1e4
FOLDS WITH MODELS:  ['fd000']


/Users/greg/Projects/flare_limits_pcnn/../sundl/sundl/dataloader/sdocml.py:193: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ).apply(lambda x: x[-1]).shift(freq = f'-{offLabel}H')#[:-int(offLabel/2)]


------------------------------
labels.shape (0,)
------------------------------


IndexError: list index out of range

In [7]:
PATH_IMAGES

PosixPath('/Users/greg/session/images')

In [10]:
pathCsvDf

NameError: name 'pathCsvDf' is not defined

In [11]:
F_PATH_TEST(configTest['labelCol'])

NameError: name 'configTest' is not defined